# Visualization

When it comes to visualizing geospatial data with/on maps with Python, a great number of tools and techniques
are available. In this lesson we will explore several of these:

* [Folium](https://github.com/python-visualization/folium)
* [ipyleaflet](https://ipyleaflet.readthedocs.io) - Interactive maps in the Jupyter notebook
* Bokeh


## Folium
Whenever you visit website that has some kind of interactive map, it
is quite probable that you are witnessing a map that has been made with
a JavaScript library called [Leaflet](http://leafletjs.com).  The
other popular library one that you might encounter is
[OpenLayers](https://openlayers.org).

There is a Python module called
[Folium](https://github.com/python-visualization/folium) that makes
it possible to visualize data that has been manipulated in Python on an
interactive Leaflet map.

### Basics
We will start with the most minimal map using the default OpenStreetMap base map.
See [Folium Quickstart](https://python-visualization.github.io/folium/quickstart.html).


In [ ]:
import folium

m = folium.Map(location=[44.43225, 26.10626])


To display it in a Jupyter notebook, simply ask for the object representation:


In [ ]:
m


You could even save this map to a file and serve it via a webserver: 

`m.save('index.html')`


### GeoJSON Overlay

It gets interesting when you can overlay the map with data manipulated
via Python. Here we overlay the map with the Polygons of all countries, though
that set is in a lower resolution clearly.


In [ ]:

countries = f'../data/countries.json'

the_map = folium.Map(
    location=[44.43225, 26.10626],
    zoom_start=2  
)

folium.GeoJson(
    countries,
    name='countries'
).add_to(the_map)

folium.LayerControl().add_to(the_map)

the_map


## Bokeh

Bokeh is a very powerful framework to produce powerful maps in combination
with data. With Geopandas and Bokeh one can produce nice looking interactive maps like in the image below:

![Bokeh and Geopandas Example](images/bokeh-example1.jpg)
*Interactive Map with Bokeh and GeoPandas - Source: [CSC L6](https://automating-gis-processes.github.io/CSC/lessons/L6/interactive-map-bokeh.html)*


### Bokeh - Links

See also:

* https://automating-gis-processes.github.io/CSC/lessons/L6/interactive-map-bokeh.html
* [Binder for Geographic Plots in Bokeh](https://mybinder.org/v2/gh/bokeh/bokeh-notebooks/master?filepath=tutorial%2F09%20-%20Geographic%20Plots.ipynb)
* https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
* https://pythonawesome.com/bokeh-plotting-backend-for-pandas-and-geopandas/


### Bokeh - Make a simple Plot
First, we learn the basic logic of plotting in Bokeh by making a simple interactive plot with a few points.

Import the necessary functionalities from Bokeh.

In [ ]:
from bokeh.plotting import figure, save


Initialize our plot by calling the `figure` object.


In [ ]:
p = figure(title="My first interactive plot!")


Next we create lists of x and y coordinates that we want to plot.


In [ ]:
x_coords = [0,1,2,3,4]
y_coords = [5,4,1,2,0]


	In Bokeh drawing points, lines or polygons are always done using 
	list(s) of x and y coordinates.

Now we can plot those as points using a `.circle()` -object. Give it a red color and size of 10.


In [ ]:
p.circle(x=x_coords, y=y_coords, size=10, color="red")


Finally, we can save our interactive plot into the disk with save -function 
that we imported in the beginning. All interactive plots are typically 
saved as html files which you can open in a web-browser.

	# Give output filepath
	outfp = r"/home/geo/points.html"
	
	# Save the plot by passing the plot -object and output path
	save(obj=p, filename=outfp)

Now you could open your interactive `points.html` plot by double-clicking it which should open it in a web browser.

But we will plot directly in the Notebook here using `output_notebook()` and `show()`.


In [ ]:
from bokeh.io import output_notebook, show
output_notebook()


And then the moment of magic:


In [ ]:
show(p)


### Bokeh - Creating an Interactive Tiled Background Map


In [ ]:

from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_notebook, show
output_notebook()


If you show the figure, you can then use the wheel zoom and pan tools to navigate over any zoom level, 
and Bokeh will request the appropriate tiles from the server and insert them at the correct locations in the plot:


In [ ]:

# When using in standard Python env
# output_file("tile.html")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)

# range bounds supplied in web mercator coordinates
p = figure(tools='pan, wheel_zoom', x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

show(p)


### Creating an Interactive Maps using Bokeh and Geopandas

Creating an interactive Bokeh map from a Shapefile or other vector data file like GeoJSON
consists typically of the following steps:

* Read the spatial vector file into `GeoDataFrame`
* Calculate the x and y coordinates of the geometries into separate columns
* Convert the `GeoDataFrame` into a Bokeh `DataSource`
* Plot the x and y coordinates as points, lines or polygons (which are in Bokeh words: `circle`, `multi_line` and `patches`)

We follow the steps below, extending and plotting on the tiled map from above.


In [ ]:

import geopandas as gpd

# Read the data (already in Web Mercator projection
points = gpd.read_file('../data/populated_places.3857.gpkg')


In [ ]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y


In [ ]:
points['x'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
points['y'] = points.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)


In [ ]:
points.head(5)


In [ ]:
p_df = points.drop('geometry', axis=1).copy()
p_df.head(2)


In [ ]:
from bokeh.models import ColumnDataSource
psource = ColumnDataSource(p_df)


In [ ]:
# p = figure(title="A map of populated places from a GeoPackage")
p.circle('x', 'y', source=psource, color='red', size=10)
show(p)


---
[<- Data Processing and Analysis](04-data-processing-analysis.ipynb) | [Metadata ->](06-metadata.ipynb)